**Задание 1.** Обязательная часть

Будем парсить страницу со свежеми новостям на https://habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [ ]:
# Литература:
# https://www.youtube.com/watch?v=ykjBVT57r68&ab_channel=%D0%90%D0%BD%D0%B4%D1%80%D0%B5%D0%B9%D0%90%D0%BD%D0%B4%D1%80%D0%B8%D0%B5%D0%B2%D1%81%D0%BA%D0%B8%D0%B9
# https://ru.stackoverflow.com/questions/1132115/%D0%9D%D0%B5-%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%D0%B5%D1%82-%D0%BF%D0%B0%D1%80%D1%81%D0%B5%D1%80-%D0%BD%D0%B0-%D0%BF%D0%B8%D1%82%D0%BE%D0%BD%D0%B5


KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all'
host = 'https://habr.com'
HEADERS = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}


import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
def get_html(url, headers=HEADERS, params=None):
    """
    Получение html странички
    """
    time.sleep(0.33)
    r = requests.get(url, headers=HEADERS, params=params)
    return r

r = get_html(URL)
r

<Response [200]>

In [ ]:

def find_elements(html, *keywords):
    """
Поиск данных о статьях по списку ключевых слов в превью по одной странице
Задавая список KEYWORDS в параметры функции, вызываю некорректную работу функции
поэтому попробовал сделать через *args
    """
    soup = BeautifulSoup(html.text)
    items = soup.find_all('article', class_='tm-articles-list__item')
    elements = []
    for el in items:
        search = 0 # поиск встречающихся KEYWORDS
        for key in keywords:
            search += el.text.lower().find(str(key).lower())+1 # .find возвращает -1 если не нашел строку
            if search > 0:
                elements.append(
                    {'date': pd.to_datetime(el.find('time').get('datetime')).date(),
                    'title': el.find('a', class_='tm-article-snippet__title-link').text,
                    'link': host + el.find('a', class_='tm-article-snippet__title-link').get('href'),
                    }
                )
                break
    return elements

In [ ]:
def max_number_page(html):
    """
Поиск наибольшей страницы
На хабре пагинация отображает последнюю страницу
Ищем на странице полоску пагинации и применяем эвристику - последний элемент items и будет max_page  
    """
    soup = BeautifulSoup(html.text)
    items = soup.find_all('a', class_='tm-pagination__page')
    max_page = int(items[-1].text)
    return max_page

html = get_html(URL)
max_number_page(html)

50

In [ ]:
html = get_html(URL)
res = []
res_df = pd.DataFrame()
for page in range(1, max_number_page(html)+1):
    url = URL+f'/page{str(page)}'
    html = get_html(url)
    res += find_elements(html, *KEYWORDS)
    if page>5:  #
        break   # Убрать для получения всех данных
pd.DataFrame(res).head()


,date,title,link
0,2021-08-17,Структура + предположения > ML? Моделирование ...,https://habr.com/ru/company/epam_systems/blog/...
1,2021-08-17,Люди python-сообщества: кто приедет на PyCon R...,https://habr.com/ru/company/it_people/blog/571...
2,2021-08-16,Трюки Pandas от RealPython,https://habr.com/ru/company/skillfactory/blog/...


**Дополнительная часть (необязательная)**

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
def search_in_text(html, *keywords):
    """
    Поиск совпадений списка keywords с текстом статьи
    search будет > 0 если есть совпадения
    """
    soup = BeautifulSoup(html.text)
    items = soup.find_all('article', class_='tm-page-article__content tm-page-article__content_inner')
    for el in items:
        search = 0
        for key in keywords:
            search += el.text.lower().find(str(key).lower())+1 # .find возвращает -1 если не нашел строку
            if search > 0:
                return True
    return False

# https://habr.com/ru/post/573580/	
# https://habr.com/ru/company/samsung/blog/573514/
html = get_html('https://habr.com/ru/post/573540')
res = search_in_text(html, *KEYWORDS)
res

True

In [ ]:
def search_all(html, *keywords):
    """
    Поиск ключевых слов keywords по тексту статьи
    Исхожу из того что вся информация для поиска есть в тексте статьи, поиск по превью не требуется
    """
    soup = BeautifulSoup(html.text)
    items = soup.find_all('article', class_='tm-articles-list__item')
    elements = []
    for i, el in enumerate(items):

        # получение html страницы текста статьи
        try:
            url = host + el.find('a', class_='tm-article-snippet__title-link').get('href')
        except: 
            print('Номер битой ссылки:', i)
            continue
        html = get_html(url) 
        soup_article_text = BeautifulSoup(html.text)

        if search_in_text(html, *keywords) == True:
            elements.append(
                {'date': pd.to_datetime(el.find('time').get('datetime')).date(),
                'title': el.find('a', class_='tm-article-snippet__title-link').text,
                'link': url,
                'text': soup_article_text.find('article', class_='tm-page-article__content tm-page-article__content_inner').text
                }
            )
    return elements


In [ ]:
# https://habr.com/ru/article/571588/ падает на этой статье
# структура поста статьи отличная от заданной в скрипте
# обошёл через конструкцию try-except
html = get_html(URL)
res = []
res_df = pd.DataFrame()
for page in range(1, max_number_page(html)+1):
    url = URL+f'/page{str(page)}'
    html = get_html(url)
    res += search_all(html, *KEYWORDS)
    print(page)
    if page>5:  #
        break   # Убрать для получения всех данных, у меня скрипт выполнялся минут 20
pd.DataFrame(res).head()


1
2
3
4
5
6


,date,title,link,text
0,2021-08-18,Первый Компьютерный ЕГЭ 2021,https://habr.com/ru/post/573580/,\n evage\n сегодня в 18:01Первый Компь...
1,2021-08-18,5 инструментов для удаленной команды Automatio...,https://habr.com/ru/post/573540/,\n ARG89\n сегодня в 15:525 инструмент...
2,2021-08-18,"Не стоит ждать, что в обозримом будущем систем...",https://habr.com/ru/company/skillfactory/blog/...,\n honyaki\n сегодня в 14:24Не стоит ж...
3,2021-08-18,"500 Dev на 10 Ops, или Как внедрить NoOps в ма...",https://habr.com/ru/company/oleg-bunin/blog/56...,\n olegbunin\n сегодня в 10:07500 Dev ...
4,2021-08-18,Перевод: Как MS SQL Server выполняет запросы. ...,https://habr.com/ru/post/573438/,\n frrrost\n сегодня в 09:50Перевод: К...


**Задание 2.** Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса https://www.avast.com/hackcheck/. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

**Подсказка:** сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

In [ ]:
# Литература:
# https://ru.stackoverflow.com/questions/1242929/%D0%9A%D0%B0%D0%BA-%D0%B7%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%B8%D1%82%D1%8C-%D0%BE%D1%82%D0%B2%D0%B5%D1%82-%D0%BD%D0%B0-post-%D0%B7%D0%B0%D0%BF%D1%80%D0%BE%D1%81-%D1%81-%D1%81%D0%B0%D0%B9%D1%82%D0%B0-%D0%B2-%D0%B2%D0%B8%D0%B4%D0%B5-json-%D1%84%D0%B0%D0%B9%D0%BB%D0%B0
# http://jsonviewer.stack.hu/
# https://pythonru.com/biblioteki/kratkoe-rukovodstvo-po-biblioteke-python-requests

import requests
import pandas as pd
EMAIL = ['xxx@x.ru', 'yyy@mail.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

PAYLOAD['emailAddresses'] = EMAIL

HEADERS = {
           'Referer': 'https://www.avast.com/',
           'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
           'sec-ch-ua-mobile': '?0',
           'Sec-Fetch-Dest': 'empty',
           'Sec-Fetch-Mode': 'cors',
           'Sec-Fetch-Site': 'same-site',
           'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
           'Vaar-Header-App-Build-Version': '1.0.0',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'}

r = requests.post(URL, headers=HEADERS, json=PAYLOAD)
r

<Response [200]>

In [ ]:
r.json()

In [342]:
breaches = pd.DataFrame(r.json()['breaches'])
breaches = breaches.T[['breachId', 'site', 'description', 'publishDate']].reset_index()
breaches = breaches[['breachId', 'site', 'description', 'publishDate']]
breaches.head()

,breachId,site,description,publishDate
0,3,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z
1,12,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
2,15,imesh.com,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z
3,2961,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da...",2017-01-31T00:00:00Z
4,3164,cfire.mail.ru,"In July and August of 2016, two criminals carr...",2017-02-14T00:00:00Z


In [ ]:
df =pd.DataFrame(r.json()['summary'])
df_ready = pd.DataFrame()
for el in EMAIL:
    list_ = list(df.loc['breaches'][el])
    df_ = pd.DataFrame(list_, columns=['breachId'])
    df_ = df_.merge(breaches, how='left')
    df_['email'] = el
    df_ready = pd.concat([df_ready, df_])
df_ready[['email', 'publishDate', 'site', 'description']].head()

,email,publishDate,site,description
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
3,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."


**Дополнительная часть** (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод https://vk.com/dev/wall.get

GROUP = 'netology’

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: 
<дата поста> - <текст поста>

In [304]:
import json
GROUP = 'netology'

WALL_GET_REQUEST = 'https://api.vk.com/method/wall.get'
NEWSFEED_REQUEST = 'https://api.vk.com/method/newsfeed.search'

with open('token.txt', 'r') as file_object:
    TOKEN = file_object.read().strip()

VERSION = '5.131'
SLEEP = 0.33

In [314]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'filter': 'owner',
    'extended': 0,
    'count': 50
}

In [315]:
response = requests.post(WALL_GET_REQUEST, params)
response

<Response [200]>

In [ ]:
response.text

In [318]:
json_obj = json.dumps(response.json())
json_obj


'{"response": {"count": 7852, "items": [{"id": 94620, "from_id": -30159897, "owner_id": -30159897, "date": 1629108009, "marked_as_ads": 0, "post_type": "post", "text": "\\u041d\\u0430\\u0448 \\u043b\\u0435\\u0442\\u043d\\u0438\\u0439 \\u0434\\u0438\\u0434\\u0436\\u0438\\u0442\\u0430\\u043b-\\u043c\\u0430\\u0440\\u0430\\u0444\\u043e\\u043d \\u043f\\u0440\\u043e\\u0434\\u043e\\u043b\\u0436\\u0430\\u0435\\u0442\\u0441\\u044f \\ud83c\\udf81\\n \\n\\u041a\\u0430\\u0436\\u0434\\u0443\\u044e \\u043d\\u0435\\u0434\\u0435\\u043b\\u044e \\u043c\\u044b \\u0431\\u0443\\u0434\\u0435\\u043c \\u0440\\u0430\\u0437\\u044b\\u0433\\u0440\\u044b\\u0432\\u0430\\u0442\\u044c \\u043f\\u0440\\u0438\\u0437\\u044b \\u043e\\u0442 \\u041d\\u0435\\u0442\\u043e\\u043b\\u043e\\u0433\\u0438\\u0438 \\u0438 \\u043d\\u0430\\u0448\\u0438\\u0445 \\u043f\\u0430\\u0440\\u0442\\u043d\\u0451\\u0440\\u043e\\u0432, \\u0430 \\u0432 \\u043a\\u043e\\u043d\\u0446\\u0435 \\u043b\\u0435\\u0442\\u0430 \\u2014 27 \\u0430\\u0432\\u0433\

In [331]:
def date_change(df):
    return datetime.utcfromtimestamp(df).strftime('%Y-%m-%d %H:%M:%S')

In [340]:
from datetime import datetime

df = pd.DataFrame(response.json()['response']['items'])[['date', 'text']]
df['data_change'] = df['date'].apply(date_change)
df[['data_change', 'text']].sort_values('data_change', ascending=False).head()

,data_change,text
1,2021-08-19 08:49:00,С появлением искусственного интеллекта тратить...
2,2021-08-18 14:00:04,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
3,2021-08-18 08:05:00,Запустили курс «Старт в SMM» 🙃\n \nSMM-специал...
4,2021-08-17 15:18:00,Вебинар ничем не отличается от любого публично...
5,2021-08-17 08:59:00,💼 20 августа в 19:00 на нашем Ютуб-канале прой...
